<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_qa_albert_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
1. [AllenNLP Q&A with heatmap](https://demo.allennlp.org/reading-comprehension/MjE0MTE2MQ==)

## Environment Setup
To get the source code of transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [0]:
!uname -a
!apt-get install file

Linux 948adb98e3a7 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Fetched 275 kB in 1s (236 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 144467 files and directories currently installed.)
Prepa

In [0]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

     |████████████████████████████████| 675kB 4.7MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 3.8MB 25.3MB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a72e720e2be4b6547cf591b9ee5c9be194d701d7a4010f3d49d0a05cbf945fdf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


In [0]:
!ls -al
!rm -rf nlp-qa
!git clone --depth 1 https://github.com/hailusong/nlp-qa.git

total 16
drwxr-xr-x 1 root root 4096 May 29 18:19 .
drwxr-xr-x 1 root root 4096 Jun  4 18:40 ..
drwxr-xr-x 1 root root 4096 Jun  2 16:14 .config
drwxr-xr-x 1 root root 4096 May 29 18:19 sample_data
Cloning into 'nlp-qa'...
fatal: could not read Username for 'https://github.com': No such device or address


In [0]:
!pip install -r nlp-qa/question-answering/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'nlp-qa/question-answering/requirements.txt'


## Configuratiuon

In [0]:
MULTILABEL_INTENT=True
QA_EXTRACTION=False

## Download

### Pre-trained ALBERT v2 language model 
Ref:
- https://huggingface.co/transformers/pretrained_models.html
- [BERT Fine-Tuning Tutorial with PyTorch](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#4-train-our-classification-model)

In [0]:
if MULTILABEL_INTENT:
  from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  from transformers import AlbertConfig, AlbertForSequenceClassification
  import torch

  config = torch.hub.load('huggingface/pytorch-transformers', 'config', 'albert-large-v2')
  config.num_labels = 3
  assert(type(config) == AlbertConfig)
  print(config)

  # Download model and configuration from S3 and cache.
  # label: lawyer letter, deed, other
  model = torch.hub.load('huggingface/pytorch-transformers', 
                         'modelForSequenceClassification', 
                         'albert-large-v2',
                         config=config
                         )
  assert(type(model) == AlbertForSequenceClassification)

  # model = torch.hub.load('huggingface/pytorch-transformers', 'model', './test/bert_model/')  # E.g. model was saved using `save_pretrained('./test/saved_model/')`
  # model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'albert-large-v2', output_attentions=True)  # Update configuration during loading
  # assert model.config.output_attentions == True


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


AlbertConfig {
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 16,
  "num_hidden_groups": 1,
  "num_hidden_layers": 24,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30000
}



Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


In [0]:
if MULTILABEL_INTENT:
  # Logging
  print('\n==== Output Layer ====\n')
  
  params = list(model.named_parameters())
  for p in params[-4:]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


==== Output Layer ====

albert.pooler.weight                                    (1024, 1024)
albert.pooler.bias                                           (1024,)
classifier.weight                                          (3, 1024)
classifier.bias                                                 (3,)


### Pre-trained ALBERT v2 model on SQuAD2
https://huggingface.co/elgeish/cs224n-squad2.0-albert-xxlarge-v1, including:
- config.json, 721.0B
- pytorch_model.bin, 849.2MB
- special_tokens_map.json, 156.0B
- spiece.model, 742.5KB
- tokenizer_config.json, 39.0B

In [0]:
if QA_EXTRACTION:
  from transformers import ( AutoTokenizer, AutoModelForQuestionAnswering )

  tokenizer = AutoTokenizer.from_pretrained("elgeish/cs224n-squad2.0-albert-xxlarge-v1")
  model = AutoModelForQuestionAnswering.from_pretrained("elgeish/cs224n-squad2.0-albert-xxlarge-v1")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [0]:
if QA_EXTRACTION:
  # inspect tokenizer max length
  print(f'Tokenizer max input length {tokenizer.model_max_length}')

Tokenizer max input length 512


## Load Training Data

### Load ALBERT multi-label training data
Some example datasets:
-  The [Corpus of Linguistic Acceptability (CoLA)](https://nyu-mll.github.io/CoLA/) dataset for single sentence classification, which is about sentences labeled as grammatically correct or incorrect - part of GLUE benchmark

In [0]:
if MULTILABEL_INTENT:
  import wget
  import os

  print('Downloading dataset...')

  # The URL for the dataset zip file.
  url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

  # Download the file (if we haven't already)
  if not os.path.exists('./cola_public_1.1.zip'):
      wget.download(url, './cola_public_1.1.zip')

In [0]:
if MULTILABEL_INTENT:
  # Unzip the dataset (if we haven't already)
  if not os.path.exists('./cola_public/'):
      !unzip cola_public_1.1.zip

In [0]:
if MULTILABEL_INTENT:
  # Get the lists of sentences and their labels.
  sentences = df.sentence.values
  labels = df.label.values

In [0]:
if MULTILABEL_INTENT:
  import pandas as pd

  # Load the dataset into a pandas dataframe.
  df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

  # Report the number of sentences.
  print('Number of training sentences: {:,}\n'.format(df.shape[0]))

  # Display 10 random rows from the data.
  df.sample(10)

In [0]:
if MULTILABEL_INTENT:
  max_len = 0

  # For every sentence...
  for sent in sentences:

      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sent, add_special_tokens=True)

      # Update the maximum sentence length.
      max_len = max(max_len, len(input_ids))

  print('Max sentence length: ', max_len)

In [0]:
if MULTILABEL_INTENT:
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 64,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', sentences[0])
  print('Token IDs:', input_ids[0])

In [0]:
if MULTILABEL_INTENT:
  from torch.utils.data import TensorDataset, random_split

  # Combine the training inputs into a TensorDataset.
  dataset = TensorDataset(input_ids, attention_masks, labels)

  # Create a 90-10 train-validation split.

  # Calculate the number of samples to include in each set.
  train_size = int(0.9 * len(dataset))
  val_size = len(dataset) - train_size

  # Divide the dataset by randomly selecting samples.
  train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

  print('{:>5,} training samples'.format(train_size))
  print('{:>5,} validation samples'.format(val_size))

In [0]:
if MULTILABEL_INTENT:
  from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

  # The DataLoader needs to know our batch size for training, so we specify it 
  # here. For fine-tuning BERT on a specific task, the authors recommend a batch 
  # size of 16 or 32.
  batch_size = 32

  # Create the DataLoaders for our training and validation sets.
  # We'll take training samples in random order. 
  train_dataloader = DataLoader(
              train_dataset,  # The training samples.
              sampler = RandomSampler(train_dataset), # Select batches randomly
              batch_size = batch_size # Trains with this batch size.
          )

  # For validation the order doesn't matter, so we'll just read them sequentially.
  validation_dataloader = DataLoader(
              val_dataset, # The validation samples.
              sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )

## Fine-tuning

### Fine-tune the ALBERT multi-label classification

In [0]:
if MULTILABEL_INTENT:
  from transformers import AdamW

  # Optimizer and LR setup
  # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
  # I believe the 'W' stands for 'Weight Decay fix"
  optimizer = AdamW(
      model.parameters(),
      lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
      eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
    )

In [0]:
if MULTILABEL_INTENT:
  from transformers import get_linear_schedule_with_warmup

  # Number of training epochs. The BERT authors recommend between 2 and 4. 
  # We chose to run for 4, but we'll see later that this may be over-fitting the
  # training data.
  epochs = 4

  # Total number of training steps is [number of batches] x [number of epochs]. 
  # (Note that this is not the same as the number of training samples).
  total_steps = len(train_dataloader) * epochs

  # Create the learning rate scheduler.
  scheduler = get_linear_schedule_with_warmup(
      optimizer, 
      num_warmup_steps = 0, # Default value in run_glue.py
      num_training_steps = total_steps)

NameError: ignored

## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


In [0]:
if QA_EXTRACTION:
  # model = torch.hub.load('huggingface/pytorch-transformers', 'modelForQuestionAnswering', 'bert-large-uncased-whole-word-masking-finetuned-squad')
  # tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-uncased-whole-word-masking-finetuned-squad')

  # The format is paragraph first and then question
  # fact = "Jim Henson was a puppeteer"
  # q = "Who was Jim Henson ?"
  fact = input('Fact: ')
  q = input('Question: ')

  # Converts a list of string to a sequence of ids (integer), using the tokenizer and vocabulary. 
  # Also request to add model-specific special tokens (such as beginning of sequence, end of sequence, sequence separator).
  indexed_tokens_fact = tokenizer.encode(fact, add_special_tokens=True)
  indexed_tokens_q = tokenizer.encode(q, add_special_tokens=True)

  # merge fact tokens and Q tokens by removing the START token in Q tokens
  indexed_tokens = indexed_tokens_fact + indexed_tokens_q[1:]
  segments_ids = [0] * len(indexed_tokens_fact) + [1] * len(indexed_tokens_q[1:])

  print(f'Length is {len(indexed_tokens)}, Indexed token: {indexed_tokens}')

  # Predict the start and end positions logits
  segments_tensors = torch.tensor([segments_ids])
  tokens_tensor = torch.tensor([indexed_tokens])

  with torch.no_grad():
      start_logits, end_logits = model(tokens_tensor, token_type_ids=segments_tensors)

  # get the highest prediction
  answer = tokenizer.decode(indexed_tokens[torch.argmax(start_logits):torch.argmax(end_logits)+1])
  print(f'The answer is: {answer}')

  # Or get the total loss which is the sum of the CrossEntropy loss for the start and end token positions (set model to train mode before if used for training)
  # start_positions, end_positions = torch.tensor([12]), torch.tensor([14])
  # multiple_choice_loss = model(tokens_tensor, token_type_ids=segments_tensors, start_positions=start_positions, end_positions=end_positions)

Fact: Robotics is an interdisciplinary branch of engineering and science that includes mechanical engineering, electrical engineering, computer science, and others. Robotics deals with the design, construction, operation, and use of robots, as well as computer systems for their control, sensory feedback, and information processing. These technologies are used to develop machines that can substitute for humans. Robots can be used in any situation and for any purpose, but today many are used in dangerous environments (including bomb detection and de-activation), manufacturing processes, or where humans cannot survive. Robots can take on any form but some are made to resemble humans in appearance. This is said to help in the acceptance of a robot in certain replicative behaviors usually performed by people. Such robots attempt to replicate walking, lifting, speech, cognition, and basically anything a human can do.
Question: What do robots that resemble humans plan  to do?
Length is 183, I